In [1]:
import torch

In [2]:
class NN():
    
    def __init__(self, inp_dim, hid_dim, out_dim):
        
        self.W, self.B = [],[]
        
        self.W.append(torch.rand(inp_dim,hid_dim[0]))
        self.B.append(torch.rand(1,hid_dim[0]))
        
        for i in range(1,len(hid_dim)):
            self.W.append(torch.rand(hid_dim[i-1], hid_dim[i]))
            self.B.append(torch.rand(1, hid_dim[i]))
        
        self.W.append(torch.rand(hid_dim[-1],out_dim))
        self.B.append(torch.rand(1,out_dim))
        
    def PRelu(self, t, alpha=0.01, grad=False):
        
        if(grad):
            return ((torch.abs(t)+t)/(2*t)) + alpha*((torch.abs(t)-t)/(2*t))
        return ((torch.abs(t)+t)/2)+(-alpha*(torch.abs(t)-t)/2)
        
    def forward(self, i_data):
        
        self.L = []
        
        self.L.append(self.PRelu(torch.add(torch.matmul(i_data,self.W[0]),self.B[0]), alpha=0.2))
        
        for i in range(1,len(self.W)):
            self.L.append(self.PRelu(torch.add(torch.matmul(self.L[i-1],self.W[i]),self.B[i]), alpha=0.2))
            
    def backward(self, i_data, o_data, lr=0.001):
        
        self.L_del,self.L_err = [],[]
        
        n = len(self.W)-1
        self.L_err.append(self.L[-1]-o_data)
        self.L_del.append(self.L_err[0]*self.PRelu(self.L[-1], grad=True))
        
        for i in range(0,n):
            self.L_err.append(torch.matmul(self.L_del[i],torch.t(self.W[n-i])))
            self.L_del.append(self.L_err[i+1] * self.PRelu(self.L[n-i-1], grad=True))
            
        self.W[0] += lr*torch.matmul(torch.t(i_data),self.L_del[n])
        self.B[0] += lr*self.L_del[n]

In [3]:
model = NN(3,[3,4],1)

In [4]:
i_data = torch.rand(1,3)
o_data = torch.rand(1)

In [5]:
model.forward(i_data)